In [50]:
import torch.nn as nn
import torch
from PIL import Image
import numpy as np
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.autograd import Variable
import os

In [51]:
# Step 1
# Get the image and change every image to an 256-dimention vector
dataSet = np.zeros([10, 256])
for i in range(0, 10):
    inputImageDir = './input/' + str(i) + '.png'
    inputImage = Image.open(inputImageDir)
    inputImage = inputImage.convert("1")
    inputImage.save(inputImageDir)
    data = inputImage.getdata()
    array = np.array(data)/255
    dataSet[i] = array
dataSet = np.array(dataSet)

In [52]:
# Step 2
# define a neural network
class Perceptron(nn.Module):

    def __init__(self, input_size, num_hidden, num_classes):
        super(Perceptron, self).__init__()
        self.inputlayer = nn.Linear(input_size, num_hidden)
        self.hiddenlayer = nn.Linear(num_hidden, num_hidden)
        self.outputlayer = nn.Linear(num_hidden, num_classes)
        self.activate = nn.Sigmoid()

    def forward(self, x):
        res = self.inputlayer(x)
        res = self.activate(res)
        res = self.hiddenlayer(x)
        res = self.activate(res)
        res = self.outputlayer(x)
        res = self.activate(res)
        return res

In [53]:
class DigitDataset(Dataset):
    def __init__(self, dataset, label_list):
        self.dataset = dataset
        self.label_list = label_list
    def __len__(self):
        return len(self.label_list)
    def __getitem__(self, idx):
        data = self.dataset[idx]
        label = self.label_list[idx]
        return {
            'data': torch.from_numpy(data).float(),
            'label': torch.from_numpy(label).float()
        }

In [54]:
# Parameters of training
input_size = 256
num_classes = 256
num_hidden = 256
learning_rate = 0.001
batch_size = 10
num_epochs = 600

# Load the dataset
train_dataset = DigitDataset(dataset = dataSet, label_list = dataSet)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=False)
device = torch.device('cpu')
model = Perceptron(input_size=input_size, num_hidden=num_hidden, num_classes=num_classes).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [55]:
# Step 3
def train(dataloader, model, num_epochs):
    for epoch in range(num_epochs):
        losses = []
        ERROR_Train = [] 
        model.train() 
        for i, data in enumerate(dataloader, 0):
            model.zero_grad()
            real_cpu, label_cpu = data['data'], data['label']
            if torch.cuda.is_available():
                real_cpu = real_cpu.cuda() 
                label_cpu = label_cpu.cuda()
            real = real_cpu
            label = label_cpu
            inputv = Variable(real)
            labelv = Variable(label)
            output = model(inputv)
            err = criterion(output, labelv) 
            err.backward() 
            optimizer.step() 

            losses.append(err.data.item())
            error = label - output.data
            ERROR_Train.extend(error)
        MAE = torch.mean(torch.abs(torch.stack(ERROR_Train)))
        ME = torch.mean(torch.stack(ERROR_Train))
        STD = torch.std(torch.stack(ERROR_Train)) 
        if epoch % 10 == 0 or epoch == num_epochs - 1:
            print('[%d/%d] Loss: %.4f MAE: %.4f Mean Error: %.4f STD: %.4f' % (epoch, num_epochs, np.average(losses), MAE, ME, STD))
    return output

# Start training        
output = train(train_loader, model, num_epochs)
# model = None
# print(output.type)

[0/600] Loss: 0.2543 MAE: 0.4866 Mean Error: 0.3988 STD: 0.3087
[10/600] Loss: 0.0430 MAE: 0.1537 Mean Error: 0.0925 STD: 0.1856
[20/600] Loss: 0.0306 MAE: 0.0907 Mean Error: 0.0345 STD: 0.1714
[30/600] Loss: 0.0282 MAE: 0.0777 Mean Error: 0.0191 STD: 0.1667
[40/600] Loss: 0.0264 MAE: 0.0726 Mean Error: 0.0157 STD: 0.1617
[50/600] Loss: 0.0249 MAE: 0.0707 Mean Error: 0.0145 STD: 0.1572
[60/600] Loss: 0.0235 MAE: 0.0688 Mean Error: 0.0133 STD: 0.1529
[70/600] Loss: 0.0223 MAE: 0.0670 Mean Error: 0.0129 STD: 0.1487
[80/600] Loss: 0.0211 MAE: 0.0655 Mean Error: 0.0122 STD: 0.1446
[90/600] Loss: 0.0199 MAE: 0.0639 Mean Error: 0.0123 STD: 0.1406
[100/600] Loss: 0.0189 MAE: 0.0624 Mean Error: 0.0118 STD: 0.1368
[110/600] Loss: 0.0178 MAE: 0.0608 Mean Error: 0.0115 STD: 0.1331
[120/600] Loss: 0.0169 MAE: 0.0592 Mean Error: 0.0113 STD: 0.1295
[130/600] Loss: 0.0160 MAE: 0.0576 Mean Error: 0.0110 STD: 0.1261
[140/600] Loss: 0.0152 MAE: 0.0561 Mean Error: 0.0107 STD: 0.1227
[150/600] Loss: 0.014

In [64]:
# Step 4
# Step 4a
# Export the image after training
# Before executing this block, create a folder called "output"
if not os.path.exists('./output'):
    os.mkdir('./output')
output_np = output.cpu().detach().numpy()
print(output_np.shape)
torch.save(model, 'net.pkl')
output_dataset = np.zeros([10, 256])
for i in range(10):
    output_img = output_np[i].reshape(16, 16)*255
    img = Image.fromarray(np.uint8(output_img))
    img = img.convert("1")
    output_path = './output/' + str(i) + '.png'
    img.save(output_path)
    data = img.getdata()
    array = np.array(data)/255
    output_dataset[i] = array

(10, 256)


In [57]:
# Step 4b
# Calculate Fh
def calculateFh(input_dataset, output_dataset):
    x, y = input_dataset.shape
    Fh_denominator = 0    # Fh分母
    Fh_numerator = 0      # Fh分子
    Fh_array = np.zeros([x])
    for j in range(x):
        for i in range(y):
            if input_dataset[j][i] == 0:
                Fh_denominator = Fh_denominator + 1
                if output_dataset[j][i] == 0:
                    Fh_numerator = Fh_numerator + 1
        Fh = Fh_numerator / Fh_denominator
        Fh_array[j] = Fh
    return Fh_array

In [58]:
# Calculate Ffa
def calculateFfa(input_dataset, output_dataset):
    x, y = input_dataset.shape
    Ffa_denominator = 0    # Ffa分母
    Ffa_numerator = 0      # Ffa分子
    Ffa_array = np.zeros([x])
    for j in range(x):
        for i in range(y):
            if input_dataset[j][i] == 1:
                Ffa_denominator = Ffa_denominator + 1
            if output_dataset[j][i] == 0 and input_dataset[j][i] == 1:
                Ffa_numerator = Ffa_numerator + 1
        Ffa = Ffa_numerator / Ffa_denominator
        Ffa_array[j] = Ffa
    return Ffa_array

In [59]:
Fh_array = calculateFh(dataSet, output_dataset)
Ffa_array = calculateFfa(dataSet, output_dataset)
print(Fh_array)
print(Ffa_array)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[0.         0.         0.         0.0010627  0.00085106 0.00071073
 0.0006105  0.0005322  0.00047438 0.00042753]


In [60]:
# TODO: Step 4c: Graph Fh as a function of Ffa for each exemplar in the input dataset

In [65]:
# Step 5
def gaussian_noise(img, mean, sigma):
    # Generate gauss noise
    noise = np.random.normal(mean, sigma, img.shape)
    # Add the noise to image
    gaussian_out = img + noise
    # Make the value between 0 and 1
    gaussian_out = np.clip(gaussian_out, 0, 1)
    return gaussian_out

gaussian_dataset = np.zeros([9, 10, 256])
std = [0.001, 0.002, 0.003, 0.005, 0.01, 0.02, 0.03, 0.05, 0.1]
if not os.path.exists('./input_noise/'):
    os.mkdir('./input_noise/')
for j in range(9):
    if not os.path.exists('./input_noise/' + str(std[j])):
        os.mkdir('./input_noise/' + str(std[j]))
    for i in range(10):
        inputImage = dataSet[i]
        gaussian_data = gaussian_noise(inputImage, 0, std[j])
        img = gaussian_data.reshape(16, 16)*255
        img = Image.fromarray(np.uint8(img))
        img.convert("1")
        inputImageDir = './input_noise/' + str(std[j]) + '/' + str(i) + '.png'
        img.save(inputImageDir)
        gaussian_dataset[j][i] = gaussian_data
gaussian_dataset = np.array(gaussian_dataset)


Fh_noise_array = np.zeros([9, 10])
Ffa_noise_array = np.zeros([9, 10])

# Train 9 datasets with noise
if not os.path.exists('./output_noise/'):
    os.mkdir('./output_noise/')
for j in range(9):
    train_noise_dataset = DigitDataset(dataset = gaussian_dataset[j], label_list = gaussian_dataset[j])
    train_noise_loader = DataLoader(dataset=train_noise_dataset, batch_size=batch_size, shuffle=False)
    print('Training dataset with noise standard deviation ' + str(std[j]))
    model_noise = torch.load('net.pkl') #  Load the model that trained before
    output_noise = train(train_noise_loader, model_noise, num_epochs)   # Train
    output_noise = model_noise(torch.from_numpy(gaussian_dataset[j]).float()) # Use the model trained before to test
    print('------------------------------------')
    output_noise_np = output_noise.cpu().detach().numpy()     # Get the output
    output_noise_dataset = np.zeros([10, 256])
#     Make the output only has 0 or 1
    if not os.path.exists('./output_noise/' + str(std[j])):
        os.mkdir('./output_noise/' + str(std[j]))
    for i in range(10):
        output_noise_img = output_noise_np[i].reshape(16, 16)*255
        img = Image.fromarray(np.uint8(output_noise_img))
        img = img.convert("1")
        output_path = './output_noise/' + str(std[j]) + '/' + str(i) + '.png'
        img.save(output_path)
        data = img.getdata()
        array = np.array(data)/255
        output_noise_dataset[i] = array
#     Calculate Fh and Ffa
    Fh = calculateFh(gaussian_dataset[j], output_noise_dataset)
    Ffa = calculateFfa(gaussian_dataset[j], output_noise_dataset)
    Fh_noise_array[j] = Fh
    Ffa_noise_array[j] = Ffa
print('------------Fh_noise_array------------')
print(Fh_noise_array)
print('------------Ffa_noise_array------------')
print(Ffa_noise_array)

Training dataset with noise standard deviation 0.001
[0/600] Loss: 0.0204 MAE: 0.0215 Mean Error: -0.0023 STD: 0.1430
[10/600] Loss: 0.0204 MAE: 0.0215 Mean Error: -0.0023 STD: 0.1430
[20/600] Loss: 0.0204 MAE: 0.0215 Mean Error: -0.0023 STD: 0.1430
[30/600] Loss: 0.0204 MAE: 0.0215 Mean Error: -0.0023 STD: 0.1430
[40/600] Loss: 0.0204 MAE: 0.0215 Mean Error: -0.0023 STD: 0.1430
[50/600] Loss: 0.0204 MAE: 0.0215 Mean Error: -0.0023 STD: 0.1430
[60/600] Loss: 0.0204 MAE: 0.0215 Mean Error: -0.0023 STD: 0.1430
[70/600] Loss: 0.0204 MAE: 0.0215 Mean Error: -0.0023 STD: 0.1430
[80/600] Loss: 0.0204 MAE: 0.0215 Mean Error: -0.0023 STD: 0.1430
[90/600] Loss: 0.0204 MAE: 0.0215 Mean Error: -0.0023 STD: 0.1430
[100/600] Loss: 0.0204 MAE: 0.0215 Mean Error: -0.0023 STD: 0.1430
[110/600] Loss: 0.0204 MAE: 0.0215 Mean Error: -0.0023 STD: 0.1430
[120/600] Loss: 0.0204 MAE: 0.0215 Mean Error: -0.0023 STD: 0.1430
[130/600] Loss: 0.0204 MAE: 0.0215 Mean Error: -0.0023 STD: 0.1430
[140/600] Loss: 0.02

[30/600] Loss: 0.0204 MAE: 0.0222 Mean Error: -0.0029 STD: 0.1428
[40/600] Loss: 0.0204 MAE: 0.0222 Mean Error: -0.0029 STD: 0.1428
[50/600] Loss: 0.0204 MAE: 0.0222 Mean Error: -0.0029 STD: 0.1428
[60/600] Loss: 0.0204 MAE: 0.0222 Mean Error: -0.0029 STD: 0.1428
[70/600] Loss: 0.0204 MAE: 0.0222 Mean Error: -0.0029 STD: 0.1428
[80/600] Loss: 0.0204 MAE: 0.0222 Mean Error: -0.0029 STD: 0.1428
[90/600] Loss: 0.0204 MAE: 0.0222 Mean Error: -0.0029 STD: 0.1428
[100/600] Loss: 0.0204 MAE: 0.0222 Mean Error: -0.0029 STD: 0.1428
[110/600] Loss: 0.0204 MAE: 0.0222 Mean Error: -0.0029 STD: 0.1428
[120/600] Loss: 0.0204 MAE: 0.0222 Mean Error: -0.0029 STD: 0.1428
[130/600] Loss: 0.0204 MAE: 0.0222 Mean Error: -0.0029 STD: 0.1428
[140/600] Loss: 0.0204 MAE: 0.0222 Mean Error: -0.0029 STD: 0.1428
[150/600] Loss: 0.0204 MAE: 0.0222 Mean Error: -0.0029 STD: 0.1428
[160/600] Loss: 0.0204 MAE: 0.0222 Mean Error: -0.0029 STD: 0.1428
[170/600] Loss: 0.0204 MAE: 0.0222 Mean Error: -0.0029 STD: 0.1428
[1

[40/600] Loss: 0.0204 MAE: 0.0251 Mean Error: -0.0055 STD: 0.1426
[50/600] Loss: 0.0204 MAE: 0.0251 Mean Error: -0.0055 STD: 0.1426
[60/600] Loss: 0.0204 MAE: 0.0251 Mean Error: -0.0055 STD: 0.1426
[70/600] Loss: 0.0204 MAE: 0.0251 Mean Error: -0.0055 STD: 0.1426
[80/600] Loss: 0.0204 MAE: 0.0251 Mean Error: -0.0055 STD: 0.1426
[90/600] Loss: 0.0204 MAE: 0.0251 Mean Error: -0.0055 STD: 0.1426
[100/600] Loss: 0.0204 MAE: 0.0251 Mean Error: -0.0055 STD: 0.1426
[110/600] Loss: 0.0204 MAE: 0.0251 Mean Error: -0.0055 STD: 0.1426
[120/600] Loss: 0.0204 MAE: 0.0251 Mean Error: -0.0055 STD: 0.1426
[130/600] Loss: 0.0204 MAE: 0.0251 Mean Error: -0.0055 STD: 0.1426
[140/600] Loss: 0.0204 MAE: 0.0251 Mean Error: -0.0055 STD: 0.1426
[150/600] Loss: 0.0204 MAE: 0.0251 Mean Error: -0.0055 STD: 0.1426
[160/600] Loss: 0.0204 MAE: 0.0251 Mean Error: -0.0055 STD: 0.1426
[170/600] Loss: 0.0204 MAE: 0.0251 Mean Error: -0.0055 STD: 0.1426
[180/600] Loss: 0.0204 MAE: 0.0251 Mean Error: -0.0055 STD: 0.1426
[

[20/600] Loss: 0.0205 MAE: 0.0326 Mean Error: -0.0117 STD: 0.1429
[30/600] Loss: 0.0205 MAE: 0.0326 Mean Error: -0.0117 STD: 0.1429
[40/600] Loss: 0.0205 MAE: 0.0326 Mean Error: -0.0117 STD: 0.1429
[50/600] Loss: 0.0205 MAE: 0.0326 Mean Error: -0.0117 STD: 0.1429
[60/600] Loss: 0.0205 MAE: 0.0326 Mean Error: -0.0117 STD: 0.1429
[70/600] Loss: 0.0205 MAE: 0.0326 Mean Error: -0.0117 STD: 0.1429
[80/600] Loss: 0.0205 MAE: 0.0326 Mean Error: -0.0117 STD: 0.1429
[90/600] Loss: 0.0205 MAE: 0.0326 Mean Error: -0.0117 STD: 0.1429
[100/600] Loss: 0.0205 MAE: 0.0326 Mean Error: -0.0117 STD: 0.1429
[110/600] Loss: 0.0205 MAE: 0.0326 Mean Error: -0.0117 STD: 0.1429
[120/600] Loss: 0.0205 MAE: 0.0326 Mean Error: -0.0117 STD: 0.1429
[130/600] Loss: 0.0205 MAE: 0.0326 Mean Error: -0.0117 STD: 0.1429
[140/600] Loss: 0.0205 MAE: 0.0326 Mean Error: -0.0117 STD: 0.1429
[150/600] Loss: 0.0205 MAE: 0.0326 Mean Error: -0.0117 STD: 0.1429
[160/600] Loss: 0.0205 MAE: 0.0326 Mean Error: -0.0117 STD: 0.1429
[17

[30/600] Loss: 0.0236 MAE: 0.0590 Mean Error: -0.0352 STD: 0.1496
[40/600] Loss: 0.0236 MAE: 0.0590 Mean Error: -0.0352 STD: 0.1496
[50/600] Loss: 0.0236 MAE: 0.0590 Mean Error: -0.0352 STD: 0.1496
[60/600] Loss: 0.0236 MAE: 0.0590 Mean Error: -0.0352 STD: 0.1496
[70/600] Loss: 0.0236 MAE: 0.0590 Mean Error: -0.0352 STD: 0.1496
[80/600] Loss: 0.0236 MAE: 0.0590 Mean Error: -0.0352 STD: 0.1496
[90/600] Loss: 0.0236 MAE: 0.0590 Mean Error: -0.0352 STD: 0.1496
[100/600] Loss: 0.0236 MAE: 0.0590 Mean Error: -0.0352 STD: 0.1496
[110/600] Loss: 0.0236 MAE: 0.0590 Mean Error: -0.0352 STD: 0.1496
[120/600] Loss: 0.0236 MAE: 0.0590 Mean Error: -0.0352 STD: 0.1496
[130/600] Loss: 0.0236 MAE: 0.0590 Mean Error: -0.0352 STD: 0.1496
[140/600] Loss: 0.0236 MAE: 0.0590 Mean Error: -0.0352 STD: 0.1496
[150/600] Loss: 0.0236 MAE: 0.0590 Mean Error: -0.0352 STD: 0.1496
[160/600] Loss: 0.0236 MAE: 0.0590 Mean Error: -0.0352 STD: 0.1496
[170/600] Loss: 0.0236 MAE: 0.0590 Mean Error: -0.0352 STD: 0.1496
[1

In [ ]:
# TODO: Step 6: Display Data from your Tests in Step 5